# Denoising Diffusion Probabilistic Models
Oğuzhan Ercan - x.com/oguzhannercan


In this chapter, we will be studying on diffusion models, more spesifically, DDPM. We will discuss what is diffusion models and why we model visual data with these models.


Diffusion models are a class of generative models that generate data by learning to reverse a process that gradually adds noise to data. The idea is to start with a data distribution (e.g., images) and progressively corrupt it with noise until it resembles a simple distribution (like Gaussian noise). Then, the model learns to reverse this process, starting from noise and reconstructing the original data distribution. 

Diffusion models operate in two phases: the forward process (adding noise) and the reverse process (removing noise).


### Forward Process (Noising)
The forward process takes a data sample $x_0$ (e.g., an image) from the true data distribution $p(x_0)$ and gradually adds **Gaussian noise** over $T$ time steps until the data resembles pure noise $x_T \sim \mathcal{N}(0,I)$. The forward process is fixed and doesn't require training. It defines a sequence of distributions $q(x_1), q(x_2), \dots, q(x_T)$ transforming $p(x_0)$ into a simple noise distribution. It's like the encoder of a **Variational Autoencoder (VAE)**, but the variational posterior is predefined. Since each step only depends on the previous step, and the process is random, it is a **Markov chain**.

![Forward Process](media/forward.png)

### Reverse Process (Denoising) 
The reverse process starts from noise $x_T \sim \mathcal{N}(0,I)$ and learns to reconstruct the original data $x_0$. It's also a Markov chain, defined by:

$p_\theta(x_{t-1} \mid x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t,t), \Sigma_\theta(x_t,t))$

where $\mu_\theta$ and $\Sigma_\theta$ are parameterized by a neural network (e.g., a U-Net) with parameters $\theta$. The model is trained to minimize the difference between the true reverse process $q(x_{t-1} \mid x_t)$ and the learned reverse process $p_\theta(x_{t-1} \mid x_t)$. This is done using **KL divergence**, leading to an **Evidence Lower Bound (ELBO)** similar to **VAEs**.

![Reverse Process](media/reverse.png)

It is often confusing to listen to the basics of diffusion model theory from beginning to end. Therefore, we will start directly with the implementation and explain the relevant parts. First we will implement the model which we refer before as ' ... are parameterized by a neural network (e.g., a U-Net)'. Since the overall pipeline is depending on time stamps t, we will use positional encoding to inject that information. Other parts are will be just basic U-Net implementation. For clear reading, we will directly import them, you can check the relative files.

In [4]:
import torch
from models.unet import UNet
model = UNet()
x = torch.randn(4, 3, 64, 64)  # B, C, H, W
t = torch.randint(0, 1000, (4,))
out = model(x, t)
print(out.shape)  # Should print torch.Size([4, 3, 64, 64])

torch.Size([4, 3, 64, 64])


Now we will create dataset, optimizer and other training related basics.

In [9]:
from data.get_dataloader import get_dataloaders
from main import load_config
config = load_config("configs/config.yaml")
train_loader, val_loader = get_dataloaders(config) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = config.training.device


First batch shape: torch.Size([64, 3, 64, 64])


Now we will dive into the DDPM, as we discussed before, we will implement 2 different algorithms, forward process (adding noise) and reverse process (removing the noise). Below we will dive into theorical aspect of forward and reverse process, if you do not understand, do not worry about it. You can follow video tutorials at the end of this notebook.

### Forward Process

Diffusion models operate by modeling a gradual process of adding and then reversing noise to data. They don't rely on an encoder-decoder setup like VAEs. Instead, they define a forward process that incrementally adds noise to the data over many steps, transforming it into pure noise, and a reverse process that learns to denoise step-by-step to recover the data.

The forward process in a diffusion model is a Markov chain that gradually adds Gaussian noise to the data point $x_0$ (e.g., an image) over $T$ steps, transforming it into isotropic Gaussian noise $x_T$. At each step $t$, a small amount of Gaussian noise with variance $\beta_t$ is added:

$$x_t = \sqrt{1 - \beta_t} x_{t-1} + \sqrt{\beta_t} \epsilon_t, \quad \epsilon_t \sim \mathcal{N}(0, I)$$

The **noise schedule** $\{\beta_t\}_{t=1}^T$ is typically pre-defined and increases gradually, ensuring that each step adds a small, manageable amount of noise. It can be also learned, constant or follows a function like cosine. By the end of the process ($t=T$), $x_T$ is nearly pure noise. 

At forward process, we somehow predefine the prior distrubiton $ p(x_T) \approx \mathcal{N}(0, I) $ (fixed, as the noise distribution at the end of the forward process) which we learn it at VAEs.

So, lets set T = 1000, which means we will add noise to clean image 1000 times and we will get noise $  \approx \mathcal{N}(0, I) $. We will be controlling how much noise added to image at each step with $ \beta $. For simplicity, we will use linear scheduler for betas. What this means is, we will start adding small noises, then we will be adding more noise. 

